# 在PYNQ上加速卷积神经网络——以手写数字识别为例

CNN_IOT是python中的一个轻量级深度学习框架，用于在IOT设备中运行卷积神经网络。<br>

此框架支持CNN的主要功能 包括：

* 卷积层(Convolutional Layer)
* 池化层(Pooling Layer)
* 全连接层(Fully connected layer) 

本UserGuide将由Lenet-5网络模型为例，通过已经训练好的参数进行手写数字识别并且对比FPGA-CPU的性能差异

### 步骤1：加载相关库

In [1]:
import time
import numpy as np
import CNNIOT
from PIL import Image

### 步骤2：读取卷积神经网络参数

注：此处采用mnist手写数字识别数据集

In [2]:
conv1_w=np.load('LeNet-weights/Conv1_weight.np.npy')
conv1_b=np.load('LeNet-weights/Conv1_bias.np.npy')
conv2_w=np.load('LeNet-weights/Conv2_weight.np.npy')
conv2_b=np.load('LeNet-weights/Conv2_bias.np.npy')
f1_w=np.load('LeNet-weights/fc1_weight.np.npy')
f2_w=np.load('LeNet-weights/fc2_weight.np.npy')
f1_b=np.load('LeNet-weights/fc1_bias.np.npy')
f2_b=np.load('LeNet-weights/fc2_bias.np.npy')
data=np.load('Mnist/mnist_data.npy')
target1=np.load('Mnist/mnist_label.npy')

### 步骤3：定义卷积神经网络结构并加载神经网络参数

![avatar](../image/LeNet-5.png)

上图为Lenet-5的网络模型结构，Lenet-5网络模型主要分为以下几个部分：
* C1(Convolutional Layer1) 卷积层1
* S2(Subsampling Layer2) &nbsp;下采样层2 
* C3(Convolutional Layer1) 卷积层3
* S4(Subsampling Layer4) &nbsp;下采样层4 <br>



注：S2与S4都是采用最大池化进行下采样，因此只定义一个通用的池化层pool1

In [3]:
conv1=CNNIOT.Convolution2D(1,10,5,5,1,1,0,0,conv1_w,conv1_b,1000000)
conv2=CNNIOT.Convolution2D(10,20,5,5,1,1,0,0,conv2_w,conv2_b,1000000)
pool1=CNNIOT.Pool(2,2,2,2,'Max',0,1,10000)
fc1=CNNIOT.FC(320,50,1,f1_w,f1_b)
fc2=CNNIOT.FC(50,10,1,f2_w,f2_b)

### 步骤4：定义训练过程中的参数

In [4]:
total = 0
correct = 0
data2 = data[0:300]
target=target1[0:300]
size = data2.shape

### 步骤5：在PL端部署LeNet-5网络模型

In [5]:
def test_PL(testnumber):
    total = 0
    correct = 0
    data2 = data[0:testnumber]
    target=target1[0:testnumber]
    size = data2.shape
    t1=time.time()
    rs = np.zeros((size[0],10))
    for i0 in range(0, size[0]):
        x=conv1.forward(data2[i0],CNNIOT.dma)
        x= pool1.forward(x,CNNIOT.dma)
        x=conv2.forward(x,CNNIOT.dma)
        x= pool1.forward(x,CNNIOT.dma)
        x=fc1.forward(x,CNNIOT.dma)
        rs[i0]=fc2.forward(x,CNNIOT.dma)

    for i in range(0, size[0]):
        if np.argmax(rs[i])==target[i]:
#             print(np.argmax(rs[i]),target[i])
            correct+=1
        total+=1
    t2=time.time()
    print ('accuracy on PL =',float(correct)/float(total))
    print('Inference Time on PL:',t2-t1)    

### 步骤6：在PS端部署LeNet-5网络模型

In [6]:
def test_PS(testnumber):        
        total = 0
        correct = 0
        data2 = data[0:testnumber]
        target=target1[0:testnumber]
        size = data2.shape
        t1=time.time()
        #CONV1 (10,5,5)
        rs = np.zeros((size[0], 10, 24, 24))
        for i0 in range(0, size[0]):
            for i in range(0, 10):
                for i1 in range(0, 24):
                    for i2 in range(0, 24):
                        tmp = data2[i0, 0, i1:i1 + 5, i2:i2 + 5]
                        rs[i0][i][i1][i2] = np.sum(np.multiply(tmp, conv1_w[i])) + conv1_b[i]
        #POOLING
        rs2 = np.zeros((size[0], 10, 12, 12))
        for i0 in range(0, size[0]):
            for i in range(0, 10):
                i1=0
                while i1<24:
                    i2=0
                    while i2<24:
                        tmp = rs[i0, i, i1:i1 + 2, i2:i2 + 2]
                        tmp=np.max(tmp)
        #RELU
                        if tmp>0:
                            rs2[i0][i][int(i1/2)][int(i2/2)] =tmp
                        i2+=2
                    i1+=2
        #conv2 (20,5,5)
        rs = np.zeros((size[0], 20, 8, 8))
        for i0 in range(0, size[0]):
            for i in range(0, 20):
                for i1 in range(0, 8):
                    for i2 in range(0, 8):
                        tmp = rs2[i0,:, i1:i1 + 5, i2:i2 + 5]
                        rs[i0][i][i1][i2] = np.sum(np.multiply(tmp, conv2_w[i])) + conv2_b[i]
        #POOLING
        rs2 = np.zeros((size[0], 20, 4, 4))
        for i0 in range(0, size[0]):
            for i in range(0, 20):
                i1=0
                while i1<8:
                    i2=0
                    while i2<8:
                        tmp = rs[i0, i, i1:i1 + 2, i2:i2 + 2]
                        tmp=np.max(tmp)
        #Relu
                        if tmp>0:
                            rs2[i0][i][int(i1/2)][int(i2/2)] =tmp
                        i2+=2
                    i1+=2
        #Flatteren
        rs = np.zeros((size[0],320))
        #FC1 320 --->50
        for i0 in range(0, size[0]):
            tmp=rs2[i0]
            rs[i0]=tmp.flatten()
        rs2 = np.zeros((size[0], 50))
        for i0 in range(0,size[0]):
            tmp=rs[i0]
            tmp=np.add(np.dot(f1_w, tmp), f1_b)
            for i in range(0,50):
        #RELU
                if tmp[i]>0:
                    rs2[i0][i]=tmp[i]
        #FC2 50 ---> 10
        rs = np.zeros((size[0], 10))
        for i0 in range(0,size[0]):
            tmp=rs2[i0]
            rs[i0]=np.add(np.dot(f2_w, tmp), f2_b)
        # LOG SOFTMAX
        rs=rs
        for i in range(0, size[0]):
            if np.argmax(rs[i])==target[i]:
                #print(np.argmax(rs[i]),target[i])
                correct+=1
            total+=1
        t2=time.time()
        print ('accuracy on PS=',float(correct)/float(total))
        print('Inference Time on PS:',t2-t1)

### 步骤7：运行PS与PL端的模型，比较运行时间

In [7]:
test_PL(10)
test_PS(10)

accuracy on PL = 1.0
Inference Time on PL: 1.5262606143951416
accuracy on PS= 1.0
Inference Time on PS: 13.270404815673828


## 附录1：CNN_IOT各模块的使用方法

### 卷积层(Convolution2D)的使用方法<br>
1.设置卷积层参数并将训练好的模型权重读入
```python
    conv = CNNIOT.Convolution2D(In.Planes, Out.Planes, Filter.H,
                                Filter.W ,Stride.H, Stride.W, 
                                Padding,Relu,Weight, Bias,Precision)
```

2.设置前向传播通道输出卷积结果
```python
    Output = Conv.forward(data,CNNIOT.dma,Load.Input)
```
3.各参数含义
* In.Planes: 输入张量的通道数
* Out.Planes: 输出张量的通道数
* Filter.H & Filter.W: 设置卷积核的大小
* Stride size: 设置卷积核的步长
* Padding: 设置填充的张量
* Relu: 设置激活函数（如果为1，那么会添加激活函数）
* Weight: 从训练好的模型中读取权重
* Bias: 从训练好的模型中读取偏置
* precision: 设置输入输出的精度
* Input data: 输入的张量
* CNNIOT.dma: 可选择默认的bit流中的dma或者自定义dma

### 池化层(Pool)的使用方法<br>
1.设置池化层参数
```python
    pool = CNNIOT.Pool(P.H,P.W,S.H,S.W,Pooling,Padding,Relu,Precision)

```

2.设置前向传播通道输出池化结果
```python
    Output= pool.forward(data,CNNIOT.dma)
```
3.各参数含义
* Input data:输入的张量
* Window size: 设置池化窗口大小
* Stride size: 设置池化步长大小
* Pooling: 设置池化模式（可选择最大池化/平均池化）
* Relu: 设置激活函数模式（如果为1，那么会添加激活函数）
* Padding:设置填充的张量
* precision: 设置输入输出的精度
* CNNIOT.dma: 可选择默认的bit流中的dma或者自定义dma

### 全连接层(Fully Connected Layer)的使用方法<br>
1.设置全连接层参数
```python
    FC = CNNIOT.FC(Input Size,Output Size,Relu,Weight,Bias,precision)

```

2.设置前向传播通道全连接结果
```python
    Output = FC.forward(data,CNNIOT.dma)
```
3.各参数含义
* Input Size: 输入全连接层的维度
* Output Size: 全连接层输出的维度
* Relu: 设置激活函数模式（如果为1，那么会添加激活函数）
* Weight: 从训练好的模型中读取权重
* Bias: 从训练好的模型中读取偏置
* precision: 设置输入输出的精度
* CNNIOT.dma: 可选择默认的bit流中的dma或者自定义dma

## 附录2：使用自定义图片进行识别

### 步骤一：加载自定义图片并以npy格式存为数据

In [8]:
pic = Image.open('./3.png')
pic1 = pic.convert("L")
np_pic = np.array(pic1)
np_pic = np_pic[np.newaxis,:,:]
np.save('./pic',np_pic)
pic1

### 步骤2：读取卷积神经网络参数

注：此处的数据读入应使用自定义图片生成的数据

In [9]:
data=np.load('./pic.npy')

### 步骤3：定义卷积神经网络结构并加载神经网络参数

In [10]:
conv1=CNNIOT.Convolution2D(1,10,5,5,1,1,0,0,conv1_w,conv1_b,1000000)
conv2=CNNIOT.Convolution2D(10,20,5,5,1,1,0,0,conv2_w,conv2_b,1000000)
pool1=CNNIOT.Pool(2,2,2,2,'Max',0,1,10000)
fc1=CNNIOT.FC(320,50,1,f1_w,f1_b)
fc2=CNNIOT.FC(50,10,1,f2_w,f2_b)

### 步骤4：在PL端部署前向传播通道网络

In [11]:
x=conv1.forward(data,CNNIOT.dma)
x= pool1.forward(x,CNNIOT.dma)
x=conv2.forward(x,CNNIOT.dma)
x= pool1.forward(x,CNNIOT.dma)
x=fc1.forward(x,CNNIOT.dma)
rs=fc2.forward(x,CNNIOT.dma)

### 步骤5：输出识别结果

In [12]:
print("Predition Number:",np.argmax(rs))

Predition Number: 3
